In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
df  = pd.read_csv("uber.csv") 

In [ ]:
df.head() 
df.info()
df.columns
df = df.drop(['Unnamed: 0', 'key'], axis= 1) 
df.head() 
df.shape
df.dtypes
df.info() 
df.describe()

In [ ]:
df.isnull().sum()  
df['dropoff_latitude'].fillna(value=df['dropoff_latitude'].mean(),inplace = True) 
df['dropoff_longitude'].fillna(value=df['dropoff_longitude'].median(),inplace = True) 
df.isnull().sum()  
df.dtypes 

In [ ]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime, errors='coerce')  
df.dtypes 

In [ ]:
df= df.assign(hour = df.pickup_datetime.dt.hour, 
             day= df.pickup_datetime.dt.day, 
             month = df.pickup_datetime.dt.month, 
             year = df.pickup_datetime.dt.year, 
             dayofweek = df.pickup_datetime.dt.dayofweek) 
df.head() 

In [ ]:
df = df.drop('pickup_datetime',axis=1) 
df.head() 
df.dtypes 

In [ ]:
df.plot(kind = "box",subplots = True,layout = (7,2),figsize=(15,20))
def remove_outlier(df1 , col): 
    Q1 = df1[col].quantile(0.25) 
    Q3 = df1[col].quantile(0.75) 
    IQR = Q3 - Q1 
    lower_whisker = Q1-1.5*IQR 
    upper_whisker = Q3+1.5*IQR 
    df[col] = np.clip(df1[col] , lower_whisker , upper_whisker) 
    return df1 
 
def treat_outliers_all(df1 , col_list): 
    for c in col_list: 
        df1 = remove_outlier(df , c) 
    return df1 
df = treat_outliers_all(df , df.iloc[: , 0::]) 
df.plot(kind = "box",subplots = True,layout = (7,2),figsize=(15,20)) 

In [ ]:
import haversine as hs
travel_dist = [] 
for pos in range(len(df['pickup_longitude'])): 
        long1,lati1,long2,lati2 = [df['pickup_longitude'][pos],df['pickup_latitude'][pos],df['dropoff_longitude'][pos],df['dropoff_latitude'][pos]] 
        loc1=(lati1,long1) 
        loc2=(lati2,long2) 
        c = hs.haversine(loc1,loc2)
        travel_dist.append(c) 
print(travel_dist) 
df['dist_travel_km'] = travel_dist 
df.head()

In [ ]:
df= df.loc[(df.dist_travel_km >= 1) | (df.dist_travel_km <= 130)] 
print("Remaining observastions in the dataset:", df.shape)

In [ ]:
incorrect_coordinates = df.loc[(df.pickup_latitude > 90) |(df.pickup_latitude < -90) | 
                                   (df.dropoff_latitude > 90) |(df.dropoff_latitude < -90) | 
                                   (df.pickup_longitude > 180) |(df.pickup_longitude < -180) | 
                                   (df.dropoff_longitude > 90) |(df.dropoff_longitude < -90) 
                                    ] 
df.drop(incorrect_coordinates, inplace = True, errors = 'ignore') 
df.head() 
df.isnull().sum() 
sns.heatmap(df.isnull()) #Free for null values 
corr = df.corr() #Function to find the correlation 
corr 
fig,axis = plt.subplots(figsize = (10,6)) 
sns.heatmap(df.corr(),annot = True) #Correlation Heatmap (Light values means highly correlated) 

In [ ]:
x = df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','hour','day','month','year','dayofweek','dist_travel_km']] 
y = df['fare_amount'] 

In [ ]:
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.33)

In [ ]:
from sklearn.linear_model import LinearRegression 
regression = LinearRegression() 
regression.fit(X_train,y_train) 
regression.coef_ 
regression.intercept_ 
prediction = regression.predict(X_test)
print(prediction) 
y_test

In [ ]:
from sklearn.metrics import r2_score  
r2_score(y_test,prediction) 
from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(y_test,prediction) 
MSE  
RMSE = np.sqrt(MSE) 
RMSE 
3.156187085348032

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
rf = RandomForestRegressor(n_estimators=100) #Here n_estimators means number of trees you want to build before making the prediction 
rf.fit(X_train,y_train) 
y_pred = rf.predict(X_test) 
y_pred 
array([ 5.714 , 10.285 , 12.68 , ..., 6.338 , 19.4685, 7.712 ]) 